In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

/opt/anaconda3/envs/syft/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:386: UserWarning: False is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
# syft absolute

In [3]:
# stdlib

In [4]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

SyftInfo: You have launched a development node at http://0.0.0.0:None.It is intended only for local use.

SyftInfo: You have launched a development node at http://0.0.0.0:None.It is intended only for local use.

In [5]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [6]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

SyftSuccess : User 'John Doe' successfully registered! To see users, run `[your_client].users`

```python
class UserPrivateKey:
  id: str = ad87a2e553774551af57b1bc32efa7bf

```

In [7]:
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# Create datasets

In [8]:
# third party
import numpy as np

In [9]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

Uploading: numpy-data: 100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.16it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [10]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

Uploading: numpy-data: 100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.93it/s]


# Data Scientist: make requests

In [11]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    raise RuntimeError()
    print("Computing mean...")
    return data


# compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [12]:
req = client_low_ds.code.request_code_execution(compute_mean)

In [13]:
# client_low_ds.code.compute_mean(data=data_low)

## Data Owner: Sync to high side

In [14]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [15]:
# low_state.rows._repr_html_()

In [16]:
low_state

syft.service.sync.sync_state.SyncState

In [17]:
# x = low_state.rows[0].object.status

### Diff state

In [18]:
diff_state = compare_states(low_state, high_state)

SyftWarning: UserCodes are hidden by default, and are part of the Requests. If you want to include them as separate objects, set `hide_usercode=False`

In [19]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [20]:
# diff_state.ignored_batches[1].unignore()

In [21]:
# diff_state[0].ignore()

### Resolve

In [22]:
# syft absolute

In [23]:
# batch = diff_state[0]

In [24]:
# batch.get_dependencies()[0].low_obj.service_func_name

In [25]:
w = resolve(diff_state)

In [26]:
w

In [27]:
w._sync_all()

In [28]:
# obj_diff_batch = diff_state[0]

# widget = resolve_single(obj_diff_batch)

# widget

# widget.click_sync()

### Sync second batch

In [29]:
# obj_diff_batch_request = diff_state[1]

# widget = resolve_single(obj_diff_batch_request)

# widget

# widget.click_sync()

# Run code on high side

In [30]:
req = client_high.requests[0]

In [31]:
# syft absolute

In [32]:
client_high.code

In [33]:
@sy.api_endpoint(
    path="reddit.test.schema",
    description="This endpoint allows for visualising the metadata of reddit's tables available in BigQuery.",
)
def schema_function(
    context,
) -> str:
    return "abc"


client_high.custom_api.add(schema_function)

> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py(523)set_result_to_store()
    522 
--> 523         if blob_store_result.is_err():
    524             raise SyftException(public_message=blob_store_result.message)



ipdb>  c


SyftSuccess: Endpoint successfully created.

In [34]:
client_high.custom_api.api_endpoints()

[syft.service.api.api.TwinAPIEndpointView]

In [35]:
# client_high.api.services.reddit.schema

In [36]:
# client.custom_endpoints

In [37]:
# for x in client_high.api.services._modules:
#     print(x)

In [38]:
# TODO: FIX
# client_high._fetch_api(client_high.credentials)

In [39]:
data_high = client_high.datasets[0].assets[0]

In [40]:
# client_high.code.compute_mean

In [41]:
node_high.python_node.queue_stash.partition.data

{}

In [42]:
result = client_high.code.compute_mean(data=data_high, blocking=True)

09/07/24 15:14:40 EXCEPTION LOG:

Encountered while executing compute_mean:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1848, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "<string>", line 7, in user_func_compute_mean_a6078e35002b688b1271721bdecd889c5694d17aed920075d3c05ab68578b77d_fa7df06862c74b556e580f4a1e6b6d8833efb1690b374352290dbffc0a8b411b
  File "<string>", line 4, in compute_mean
RuntimeError


    2  def compute_mean(data) -> float:
--> 3      raise RuntimeError()
    4      print('Computing mean...')

09/07/24 15:14:41 EXCEPTION LOG:

Encountered while executing compute_mean:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1848, in execute_byte_code
    result = eval(evil_string, _global

> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py(523)set_result_to_store()
    522 
--> 523         if blob_store_result.is_err():
    524             raise SyftException(public_message=blob_store_result.message)



ipdb>  blob_store_result


ipdb>  c


In [43]:
result

TwinPointer(Real):
SyftError: Exception encountered while running compute_mean, please contact the Node Admin for more info.
Encountered while executing compute_mean:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1848, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "<string>", line 7, in user_func_compute_mean_a6078e35002b688b1271721bdecd889c5694d17aed920075d3c05ab68578b77d_fa7df06862c74b556e580f4a1e6b6d8833efb1690b374352290dbffc0a8b411b
  File "<string>", line 4, in compute_mean
RuntimeError


    2  def compute_mean(data) -> float:
--> 3      raise RuntimeError()
    4      print('Computing mean...')

In [ ]:
job = client_high.requests[0].deposit_result(result)

In [ ]:
code = client_high.code[0]

In [44]:
job

syft.service.job.job_stash.Job

In [42]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)

In [43]:
# job_high

In [44]:
display(job_high)

syft.service.job.job_stash.Job

In [ ]:
# wait for the result
job_high.wait().get()

In [ ]:
job_high

In [54]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [55]:
job_info

syft.service.job.job_stash.JobInfo

In [58]:
# accept_res

In [59]:
# accept_res = request.accept_by_depositing_result(job_info)

# assert isinstance(accept_res, SyftSuccess)
# accept_res

## Assertions

In [63]:
# # Need to refresh Job because we overwrite result
# job_high = client_high.jobs[0]

# action_store_high = node_high.python_node.get_service("actionservice").store
# blob_store_high = node_high.python_node.get_service(
#     "blobstorageservice"
# ).stash.partition
# # assert (
# #     f"{client_low_ds.verify_key}_READ"
# #     in action_store_high.permissions[job_high.result.id.id]
# # )
# # assert (
# #     f"{client_low_ds.verify_key}_READ"
# #     in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
# # )

In [64]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [65]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [66]:
high_state

syft.service.sync.sync_state.SyncState

In [67]:
diff_state_2 = compare_states(high_state, low_state)

SyftWarning: UserCodes are hidden by default, and are part of the Requests. If you want to include them as separate objects, set `hide_usercode=False`

In [68]:
diff_state_2

syft.service.sync.diff_state.NodeDiff

In [70]:
w = resolve(diff_state_2)

In [71]:
w

In [73]:
w._share_all()

In [74]:
w._sync_all()

Decision: Syncing 5 objects
Decision: Syncing 5 objects
Decision: Syncing 1 objects


### Syc code

In [75]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [76]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

# Run code low

## Run

In [77]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

diff_state = compare_states(low_state, high_state)

SyftWarning: UserCodes are hidden by default, and are part of the Requests. If you want to include them as separate objects, set `hide_usercode=False`

In [78]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [81]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

SyftWarning: Loading results from cache.

```python
TwinPointer(Real)
array([15, 16, 17, 18, 19])
```


In [83]:
res_low.get()

array([15, 16, 17, 18, 19])

In [89]:
# code = client_low_ds.code[0]

# assert res_low.get().mean() == private_high.mean()
# # assert (
# #     res_low.id.id
# #     == job_high.result.id.id
# #     == code.output_history[-1].output_ids[0].id.id
# # )
# assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [91]:
# res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
TwinPointer(Real)
array([15, 16, 17, 18, 19])
```


In [92]:
# code = client_low_ds.code[0]

# assert res_low.get() == private_high.mean()
# assert (
#     res_low.id.id
#     == job_high.result.id.id
#     == code.output_history[-1].output_ids[0].id.id
# )
# assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [93]:
private_high.mean()

17.0

In [94]:
# job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
# job_low

In [95]:
# job_low.wait().get()

In [61]:
job_low.logs()

Computing mean...



In [96]:
# assert job_low.id == job_high.id
# assert job_low.result.id == job_high.result.id
# assert (
#     job_low.result.syft_blob_storage_entry_id
#     == job_high.result.syft_blob_storage_entry_id
# )

In [97]:
# job_high